## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from time import time, sleep
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from IPython.core.display import display, HTML

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 500
NUMBER_OF_ACTIVATED_PARTICIPANTS = 40
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']
DISPLAY_TX_EXECUTION = False
TEST_ACCOUNT_ADDRESS = 'bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0'
TEST_ACCOUNT_SEED = 'end spread blind steak install glare pride pony switch exit zone faint opinion march layer illness can nest fence top debate monitor supreme noble'

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(43)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

PARTICIPANTS_FILE_NAME = 'participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in tqdm(range(NUMBER_OF_PARTICIPANTS)):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

  0%|          | 0/500 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1000000,john16495701710,QmV25axme4qJsqbYMoYZx9GQHXT25WHWBZYJ7YNUCmMpbf,0xd1f01c98c6361b15c7176a08f248da306370fdd7,0x3a751d5df5c358f471edcc21c25d5f43b6dc3c37444d...,bostrom1dth5c37g5zdq7mmxfr8l9sheyz35xmweqe5q02,cosmos1dth5c37g5zdq7mmxfr8l9sheyz35xmwer2qn3d,noodle prosper reflect donor bright inquiry co...,bostrom1dth5c37g5zdq7mmxfr8l9sheyz35xmweqe5q02...,0x9dc5c14774bfb1d0ac9c3f61d4a274e0d220119b44e9...,{'pub_key': 'A6j9IYOmx1Jt5DSedDgVNujK1nGLD5yNG...,eyJwdWJfa2V5IjoiQTZqOUlZT214MUp0NURTZWREZ1ZOdW...
1,1000000,john16495701711,QmRdzvAKiWs4H6TpBJ3Q3QeqRAaY2wsguU1txJkPYJ7vcJ,0x35176878370e039ccb8ba09a142358aad40c0be8,0xc6b2f06c863cb25e56499bc6f42a1543f8cecdf68072...,bostrom1wmapx0neuech3yw4ts3s75e7qck2ndha2tnn6v,cosmos1wmapx0neuech3yw4ts3s75e7qck2ndhafc8qyt,they upper require oval finish kitchen clog th...,bostrom1wmapx0neuech3yw4ts3s75e7qck2ndha2tnn6v...,0x14c6650cd3df447645eaa00f60ba673d921977b79a52...,{'pub_key': 'ApLKDuuSXoLP1PqHVU34IHkv3v43PSoNs...,eyJwdWJfa2V5IjoiQXBMS0R1dVNYb0xQMVBxSFZVMzRJSG...
2,1000000,john16495701712,QmPtY3cM1umfPHfV2ZfxBh8UKEfYFgq1Gfcvji3DVnMeU7,0x89101a2024e06f71724d4dc190ef5fdbc1370cd8,0x4345c4447770e2820312a60c7d2da29da0e4b2bafc22...,bostrom1m0j0gg27c9cjphsvw5fq3j794twtaddl4e5pmq,cosmos1m0j0gg27c9cjphsvw5fq3j794twtaddlk2qj98,thank ski machine undo jewel neither replace s...,bostrom1m0j0gg27c9cjphsvw5fq3j794twtaddl4e5pmq...,0x82314aa409cffe5148f822af633c85f9f1bd0c0192b7...,{'pub_key': 'AhUCI1w0U4MSfkteLrwLM3EhUaQVWTFJ5...,eyJwdWJfa2V5IjoiQWhVQ0kxdzBVNE1TZmt0ZUxyd0xNM0...
3,1000000,john16495701713,QmRHLv4J2Z6y2BFwZMrfoF3aujBk1rjshZkdmgdSsNCc29,0x34f1174d3717ac670f6751fcc036b34fc4a892ff,0x4799e12d14b529d627b7ad608312abe3abe911e7c38f...,bostrom1nnj8ud45p4ev7chg8m870jtv67d0sa8e6fw4m9,cosmos1nnj8ud45p4ev7chg8m870jtv67d0sa8ee66x9z,woman bless wear chaos stereo grace inner firs...,bostrom1nnj8ud45p4ev7chg8m870jtv67d0sa8e6fw4m9...,0xb38bdf095218cbb8bc9c35601d25ca09170002b54f2d...,{'pub_key': 'A1jQg1Ww8e2FsilUNNTH0gyAETzFp2Dn2...,eyJwdWJfa2V5IjoiQTFqUWcxV3c4ZTJGc2lsVU5OVEgwZ3...
4,1000000,john16495701714,QmXPTPH9onvwMGECfC2a315jR5yHv1xc1AA24zo49KPDwA,0x85c4291e76ca7d3c1d5dbe19778146653e149643,0xd007d9f8286e6a49c4e1751ef67ca8462096b38d38e9...,bostrom1djhnjvl6rhfcqj60yw8sh8s8286jfsy8h3kcrj,cosmos1djhnjvl6rhfcqj60yw8sh8s8286jfsy85zzta4,neither reopen hammer jazz smile muscle split ...,bostrom1djhnjvl6rhfcqj60yw8sh8s8286jfsy8h3kcrj...,0x1d0ed6dc4c2d1c890d3d9e164aae5ca6a0380c2ff635...,{'pub_key': 'AwJh/dhAK9yJo/yJO1G9LqpnaRSd9P0dt...,eyJwdWJfa2V5IjoiQXdKaC9kaEFLOXlKby95Sk8xRzlMcX...
...,...,...,...,...,...,...,...,...,...,...,...,...
495,1000000,john1649570171495,QmR1PAbXT5yLAyFGDU1V2aiKwGKBeDTPppy4Xbtg2CaLKo,0xaf9b15039827b9368bad79abcc3114b8626cd320,0xac52e86a9d533d035f90151739db65a15d1f6795ecc8...,bostrom1rq0957kew7kv8hj4sg3h2v9zsxad6y823cy7zn,cosmos1rq0957kew7kv8hj4sg3h2v9zsxad6y82jtsdu5,bag sure walk battle rail exist sleep reject a...,bostrom1rq0957kew7kv8hj4sg3h2v9zsxad6y823cy7zn...,0xada4052d7140cbbac8e2236e0a613b0c7a2be5b2c60b...,{'pub_key': 'AqtPOHbxslsnXFkwLV8Y5r+cpfZWEST51...,eyJwdWJfa2V5IjoiQXF0UE9IYnhzbHNuWEZrd0xWOFk1ci...
496,1000000,john1649570171496,QmdSW9gUSozfPae5fcogkScvp8gpP9n4patmJDHq1JBuLr,0xba98748b4cb34b76c10440db31e89a1d88e3080f,0x64fb49296e3ec3b797f0b7c71a9c7fb539312b06e4c9...,bostrom1daedzeftlasfparuxka4qe25wckh9v8ewglz5r,cosmos1daedzeftlasfparuxka4qe25wckh9v8edmt32y,auction love lion dutch giant miracle vote sec...,bostrom1daedzeftlasfparuxka4qe25wckh9v8ewglz5r...,0x5e17c5615bec52860e55cb24c288fa55727a30d495e9...,{'pub_key': 'AixjLGYWsjJ0ZdCYdYqvpzq1oiDJPMZn1...,eyJwdWJfa2V5IjoiQWl4akxHWVdzakowWmRDWWRZcXZwen...
497,1000000,john1649570171497,QmY4dLukHTJPphV1Wdx86GkXLc9zXKJkHBGAVQnfEeZSBG,0x98668ddb798d94ce47038c9e15ce058fa3e7f96d,0x02263b35db1b757f850e4ce29cb97d13583892a55300...,bostrom1069t0hne

### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 10
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
print(tasks)
with Pool(processes=NUMBER_OF_THREADS) as pool:
    res = pool.starmap(get_proofs, tasks)
assert res == [True] * len(res)

{('root_testing_source_500_addresses.json', 'temp/proofs_0.json', 0, 1001)}
yarn run v1.22.17
$ ts-node index.ts --input root_testing_source_500_addresses.json --output temp/proofs_0.json --start_index 0 --end_index 1001
Merkle root: b40fa6d3d0ffdc7a746792841507786fc00d4e64c8a22d090c6afe08e64ab4e0
Number of addresses in the Merkle tree: 1001
Done in 2.68s.



In [5]:
roots = []
proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
for task in tasks:
    with open(task[1], 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    roots.append(root_and_proof_json['merkle_root'])
    proofs_df = proofs_df.append(pd.DataFrame(root_and_proof_json['proofs']), ignore_index=True)
assert roots == [roots[0]] * len(roots)
root = roots[0]
proofs_df

,address,amount,proof
0,0xd1f01c98c6361b15c7176a08f248da306370fdd7,1000000,[3eac5929d5872721fe1e2828d9373a1c62cfebf90cd8b...
1,0x35176878370e039ccb8ba09a142358aad40c0be8,1000000,[1579595b7b694fbc1ac5abebad21bcceb43614c7d927c...
2,0x89101a2024e06f71724d4dc190ef5fdbc1370cd8,1000000,[c560b5e2f62a4c4c889c3537b6977b2204d4af1a98254...
3,0x34f1174d3717ac670f6751fcc036b34fc4a892ff,1000000,[2cbbe2be547e8618c958a984b7c1dfedd982a5ba197ce...
4,0x85c4291e76ca7d3c1d5dbe19778146653e149643,1000000,[cfe5030a7cad4140c112a510605c171c192347aba5619...
...,...,...,...
996,cosmos1daedzeftlasfparuxka4qe25wckh9v8edmt32y,1000000,[da14fbcae7ae4c22f5d5deb5c2258cc51081c0127f99c...
997,cosmos1069t0hne60792r4lykss8nffphcw7p65pz8k6a,1000000,[0de2c489263dca24232b7d009a00f270eed06bc04f9df...
998,cosmos1p32r7340fgymndvwns3y46eys5gge5czzwadqd,1000000,[a5f816610fa05b805e2ad12edea93366f7bc60e438cd1...
999,cosmos1auqhx64tf60ccf8h9287xelke8cwpra7tqdmkg,1000000,[ea364902c359b1e2c6ab41843473482ee9adc6dcd6eb9...


In [6]:
cosmos_proofs_df = proofs_df[proofs_df.address.str.startswith('cosmos')][['address', 'proof']]
ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

claims_with_proofs_df = claims_df\
                .merge(
                    ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
                    how='left',
                    on='ethereum_address')\
                .merge(
                    cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
                    how='left',
                    on='cosmos_address')

claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,1000000,john16495701710,QmV25axme4qJsqbYMoYZx9GQHXT25WHWBZYJ7YNUCmMpbf,0xd1f01c98c6361b15c7176a08f248da306370fdd7,0x3a751d5df5c358f471edcc21c25d5f43b6dc3c37444d...,bostrom1dth5c37g5zdq7mmxfr8l9sheyz35xmweqe5q02,cosmos1dth5c37g5zdq7mmxfr8l9sheyz35xmwer2qn3d,noodle prosper reflect donor bright inquiry co...,bostrom1dth5c37g5zdq7mmxfr8l9sheyz35xmweqe5q02...,0x9dc5c14774bfb1d0ac9c3f61d4a274e0d220119b44e9...,{'pub_key': 'A6j9IYOmx1Jt5DSedDgVNujK1nGLD5yNG...,eyJwdWJfa2V5IjoiQTZqOUlZT214MUp0NURTZWREZ1ZOdW...,[3eac5929d5872721fe1e2828d9373a1c62cfebf90cd8b...,[f05c1c3ed43f2f73c930b4077562375c55e42716cb11e...
1,1000000,john16495701711,QmRdzvAKiWs4H6TpBJ3Q3QeqRAaY2wsguU1txJkPYJ7vcJ,0x35176878370e039ccb8ba09a142358aad40c0be8,0xc6b2f06c863cb25e56499bc6f42a1543f8cecdf68072...,bostrom1wmapx0neuech3yw4ts3s75e7qck2ndha2tnn6v,cosmos1wmapx0neuech3yw4ts3s75e7qck2ndhafc8qyt,they upper require oval finish kitchen clog th...,bostrom1wmapx0neuech3yw4ts3s75e7qck2ndha2tnn6v...,0x14c6650cd3df447645eaa00f60ba673d921977b79a52...,{'pub_key': 'ApLKDuuSXoLP1PqHVU34IHkv3v43PSoNs...,eyJwdWJfa2V5IjoiQXBMS0R1dVNYb0xQMVBxSFZVMzRJSG...,[1579595b7b694fbc1ac5abebad21bcceb43614c7d927c...,[298a35ac9c7a1531a22d63608124d52ae946bf16478ba...
2,1000000,john16495701712,QmPtY3cM1umfPHfV2ZfxBh8UKEfYFgq1Gfcvji3DVnMeU7,0x89101a2024e06f71724d4dc190ef5fdbc1370cd8,0x4345c4447770e2820312a60c7d2da29da0e4b2bafc22...,bostrom1m0j0gg27c9cjphsvw5fq3j794twtaddl4e5pmq,cosmos1m0j0gg27c9cjphsvw5fq3j794twtaddlk2qj98,thank ski machine undo jewel neither replace s...,bostrom1m0j0gg27c9cjphsvw5fq3j794twtaddl4e5pmq...,0x82314aa409cffe5148f822af633c85f9f1bd0c0192b7...,{'pub_key': 'AhUCI1w0U4MSfkteLrwLM3EhUaQVWTFJ5...,eyJwdWJfa2V5IjoiQWhVQ0kxdzBVNE1TZmt0ZUxyd0xNM0...,[c560b5e2f62a4c4c889c3537b6977b2204d4af1a98254...,[fadaf81d5a751a0cd4a17c23fbfc84cc3da759d7b04f4...
3,1000000,john16495701713,QmRHLv4J2Z6y2BFwZMrfoF3aujBk1rjshZkdmgdSsNCc29,0x34f1174d3717ac670f6751fcc036b34fc4a892ff,0x4799e12d14b529d627b7ad608312abe3abe911e7c38f...,bostrom1nnj8ud45p4ev7chg8m870jtv67d0sa8e6fw4m9,cosmos1nnj8ud45p4ev7chg8m870jtv67d0sa8ee66x9z,woman bless wear chaos stereo grace inner firs...,bostrom1nnj8ud45p4ev7chg8m870jtv67d0sa8e6fw4m9...,0xb38bdf095218cbb8bc9c35601d25ca09170002b54f2d...,{'pub_key': 'A1jQg1Ww8e2FsilUNNTH0gyAETzFp2Dn2...,eyJwdWJfa2V5IjoiQTFqUWcxV3c4ZTJGc2lsVU5OVEgwZ3...,[2cbbe2be547e8618c958a984b7c1dfedd982a5ba197ce...,[ad20470a308fe5746c97802dc8785e7c0c683fc8de7c1...
4,1000000,john16495701714,QmXPTPH9onvwMGECfC2a315jR5yHv1xc1AA24zo49KPDwA,0x85c4291e76ca7d3c1d5dbe19778146653e149643,0xd007d9f8286e6a49c4e1751ef67ca8462096b38d38e9...,bostrom1djhnjvl6rhfcqj60yw8sh8s8286jfsy8h3kcrj,cosmos1djhnjvl6rhfcqj60yw8sh8s8286jfsy85zzta4,neither reopen hammer jazz smile muscle split ...,bostrom1djhnjvl6rhfcqj60yw8sh8s8286jfsy8h3kcrj...,0x1d0ed6dc4c2d1c890d3d9e164aae5ca6a0380c2ff635...,{'pub_key': 'AwJh/dhAK9yJo/yJO1G9LqpnaRSd9P0dt...,eyJwdWJfa2V5IjoiQXdKaC9kaEFLOXlKby95Sk8xRzlMcX...,[cfe5030a7cad4140c112a510605c171c192347aba5619...,[728628f60cfcd64001c2c23a523c6de169fa40feb1600...


## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6
Avatar subgraph contract address: bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq
Proof subgraph contract address: bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1g59m935w4kxmtfx5hhykre7w9q497ptp66asspz76vhgarss5ensdy35s8


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [10]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom14d067652qyney947evnlllydm9mxhg3wq4r3k585tc2dt5x2qrdsgxhm06


### Initiate Class of Output Parsing

In [11]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Register Merkle Root

In [12]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: b40fa6d3d0ffdc7a746792841507786fc00d4e64c8a22d090c6afe08e64ab4e0
Gas used: 121,492
Tx hash: 310C6BBFB39E1FD6AF4375D90030700ED4AC7A509B9DFF12805391C030697FB1


#### Get Merkle Root form the Gift Contract

In [13]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom14d067652qyney947evnlllydm9mxhg3wq4r3k585tc2dt5x2qrdsgxhm06
{'data': {'merkle_root': 'b40fa6d3d0ffdc7a746792841507786fc00d4e64c8a22d090c6afe08e64ab4e0'}}


### Send coins to new addresses

In [14]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 1_000

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in bostrom_addresses_chunks:
    send_output = contract_utils.send_coins(
            from_seed=WALLET_SEED,
            to_addresses=bostrom_addresses_item,
            amounts=[1] * len(bostrom_addresses_item),
            gas=min(100_000 + 25_000 * len(bostrom_addresses_item), int(24e6)),
            display_data=DISPLAY_TX_EXECUTION)
    contract_utils.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom1dth5c37g5zdq7mmxfr8l9sheyz35xmweqe5q02
	amount: 1boot
	receiver: bostrom1wmapx0neuech3yw4ts3s75e7qck2ndha2tnn6v
	amount: 1boot
	receiver: bostrom1m0j0gg27c9cjphsvw5fq3j794twtaddl4e5pmq
	amount: 1boot
	receiver: bostrom1nnj8ud45p4ev7chg8m870jtv67d0sa8e6fw4m9
	amount: 1boot
	receiver: bostrom1djhnjvl6rhfcqj60yw8sh8s8286jfsy8h3kcrj
	amount: 1boot
	receiver: bostrom1lv6x3yfzj3fx6cjgruqa8s2mjruptugtleetmf
	amount: 1boot
	receiver: bostrom1aadd4jt2cl6unmjt8yvcgq8wcksqaftqpyy4mj
	amount: 1boot
	receiver: bostrom149h0exg24jn762y9lmv3jqese99kre2n85pex5
	amount: 1boot
	receiver: bostrom1rqp8smjcr6xrpq49y8akuxlx7a0zhavc2nneul
	amount: 1boot
	receiver: bostrom1pnkvcx2wm8t3x9n6sa34f6zlrkvtj4g3ntj6wx
	amount: 1boot
	receiver: bostrom1g4vq7rvxlmvt7xu42m88drvg6z25vvsdgwez0u
	amount: 1boot
	receiver: bostrom16yqfnptnzywemafzan9mwkv5me7k655dwh36pk
	amount: 1boot
	receiver: bostrom1e0pzyst8pskh7s3zzdrwc0tlxf9tdzh9qj0q9y
	amount: 1boot
	receiver: bostrom1nzu2dvvxj

### Create Passports, Proof Addresses and Claim Gift

In [16]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py {PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 20

tasks = [[item] for item in range(NUMBER_OF_ACTIVATED_PARTICIPANTS)]
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')

with Pool(processes=NUMBER_OF_THREADS) as pool:
    res = pool.starmap(execute_participation, tasks)

Number of tasks: 40


In [17]:
with open('temp/contract_execution_log_10.txt', 'r') as f:
    data = json.load(f)

In [18]:
contract_utils.parse_contract_execution_json(data['create'])


Events

cyberlinks
	QmemPmMC2CTtAFhDmwvjs5uGCVTfrf3RSdRDjAiF4AeeaJ -> Qma1WXgcUeKRooe5ydVLptoY5rRogZA7yVuBb8ftoARENc
	Qma1WXgcUeKRooe5ydVLptoY5rRogZA7yVuBb8ftoARENc -> QmemPmMC2CTtAFhDmwvjs5uGCVTfrf3RSdRDjAiF4AeeaJ
	neuron: Name Subgraph Contract

	Qma1WXgcUeKRooe5ydVLptoY5rRogZA7yVuBb8ftoARENc -> QmeMj59L9JW8QNVqL9TsVDmxTEhTDsV6q6XAH3T71aVHhC
	QmeMj59L9JW8QNVqL9TsVDmxTEhTDsV6q6XAH3T71aVHhC -> Qma1WXgcUeKRooe5ydVLptoY5rRogZA7yVuBb8ftoARENc
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1g4vq7rvxlmvt7xu42m88drvg6z25vvsdgwez0u wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 1384
Gas used: 440,265
Tx hash: E0B0F94C1835A04419B5EA7BFBE95B15553061B170AA13B68DEB5933B288E82C


In [19]:
contract_utils.parse_contract_execution_json(data['proof_ethereum'])


Events

cyberlinks
	Qma1WXgcUeKRooe5ydVLptoY5rRogZA7yVuBb8ftoARENc -> QmYfHHqWxi7PrGuXhDsTXPBdrAPh5BnWR7MkqfsAN3priG
	QmYfHHqWxi7PrGuXhDsTXPBdrAPh5BnWR7MkqfsAN3priG -> Qma1WXgcUeKRooe5ydVLptoY5rRogZA7yVuBb8ftoARENc
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom1g4vq7rvxlmvt7xu42m88drvg6z25vvsdgwez0u wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john164957017110
	address: 0x51126745fe2b70bf42f329e5c451c6c2e067d61c
Gas used: 281,156
Tx hash: 66ADD954E5CB27059B18593EEF566DC3F2B3FA70977949A1C736F3ACD6C64441


In [20]:
contract_utils.parse_contract_execution_json(data['claim_ethereum'])


Events

coin received
	receiver: bostrom1g4vq7rvxlmvt7xu42m88drvg6z25vvsdgwez0u
	amount: 100000boot

coin spent
	spender: Gift Contract
	amount: 100000boot

execute
	execute contract: Gift Contract

message from bostrom1g4vq7rvxlmvt7xu42m88drvg6z25vvsdgwez0u wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1g4vq7rvxlmvt7xu42m88drvg6z25vvsdgwez0u
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x51126745fe2b70bf42f329e5c451c6c2e067d61c
	target: bostrom1g4vq7rvxlmvt7xu42m88drvg6z25vvsdgwez0u
	amount: 12,236,691
Gas used: 322,718
Tx hash: 0381A20561A88F18A78ADB94083DA34B4E1A69802444673E7AF23E6DF8CB71D7


## Passport NFT testing
### Transfer Passport

In [21]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [22]:
for index, row in tqdm(claims_with_proofs_df[:4].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1wmapx0neuech3yw4ts3s75e7qck2ndha2tnn6v wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom1wmapx0neuech3yw4ts3s75e7qck2ndha2tnn6v
	recipient: Passport Owner Address
	token_id: 1386
Gas used: 385,800
Tx hash: FFAF3465A41FABAA47EB845FAEBE709EEC1F01C4198CFACD12F57BB978FD5881


### Burn

In [23]:
for index, row in tqdm(claims_with_proofs_df[2:6].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	cyberhole -> Nickname
	Nickname -> cyberhole
	neuron: Name Subgraph Contract

	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1m0j0gg27c9cjphsvw5fq3j794twtaddl4e5pmq wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1m0j0gg27c9cjphsvw5fq3j794twtaddl4e5pmq
	token_id: 1390
Gas used: 381,878
Tx hash: 2557A6805C6EFF0440357CF80D7F404E83BA2BEE0A82A310F41CB291A0C5D57D


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
